# OLD SECTION

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-court-suuzy")
version = project.version(6)
dataset = version.download("yolov8")

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m-pose.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="tennis-court-6/data.yaml", epochs=25, imgsz=1280,    degrees=0.0,    # rotation
    translate=0.0,  # translation
    scale=0.0,      # scaling
    shear=0.0,      # shear
    flipud=0.0,     # vertical flip
    fliplr=0.0,     # horizontal flip
    mosaic=0.0,     # mosaic augmentation
    mixup=0.0,      # mixup augmentation
    copy_paste=0.0,
    batch=8)

In [ ]:
model = YOLO("runs/pose/train11/weights/best.pt")

# test inference pada sebuah gambar
results = model("test3.jpg", save=True, show=True)

In [ ]:
model = YOLO("runs/pose/train11/weights/best.pt")

# test inference pada sebuah gambar
results = model("test3.jpg", save=True, show=True)

In [ ]:
model = YOLO("runs/pose/train12/weights/best.pt")

# test inference pada sebuah gambar
results = model.track("video1.mp4", save=True, show=True)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("runs/pose/train11/weights/best.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model("video1.mp4", stream=True)  # return a generator of Results objects

# Process results generator
for result in results:
    keypoints = result.keypoints  # Keypoints object for pose outputs
    result.save(filename="DELETE/result.jpg")

# NEW SECTION

In [2]:
import dagshub
import datetime
import mlflow
import torch
import time
import os
import shutil
import random
import yaml
from pathlib import Path
from roboflow import Roboflow
from ultralytics import YOLO, settings

In [ ]:
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-court-suuzy")
version = project.version(8)
dataset = version.download("yolov8", location="datasets/keypoints-v2")

In [3]:
dagshub.init(repo_owner='abiyamf', repo_name='courtPlay', mlflow=True)
settings.update({"mlflow": True})

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

install "ipywidgets" for Jupyter support



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d9e99fc3-5cb1-4793-a169-22d9c0a7ea33&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=551d978828fc5f290339b58ea09e1cf5f4ab8dc22e2e84cc3becab4a3dc16d0d




Accessing as rraaffllyy22

Initialized MLflow to track repo "abiyamf/courtPlay"

Repository abiyamf/courtPlay initialized!

In [6]:
# Global Config
os.environ["MLFLOW_KEEP_RUN_ACTIVE"] = "True"
time_now = datetime.datetime.now().strftime("%m-%d_%H-%M")
gpu_name = torch.cuda.get_device_name(0)
projectType = "key-points"
datasetVersion = "8"
yoloVersion = "yolo11"
modelSize = "l"
epochs = 1
imgsz = 1280
batch = 4
data_path = "datasets/keypoints-v2/data.yaml"
project_path = f"results/key-points-v2/{yoloVersion}/{modelSize}"

def format_duration(seconds: float) -> str:
    """Convert seconds into 'Hh Mm Ss' format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}h {minutes}m {secs}s"

In [5]:
# Load a model
model = YOLO(f"{yoloVersion}{modelSize}-pose.pt")

mlflow.set_experiment(f"courtPlay-{modelSize}")

start_time = time.time()

with mlflow.start_run(run_name=f"{time_now}"):
    mlflow.log_params({
        "project type": projectType,
        "dataset version": datasetVersion,
        "yolo version": yoloVersion,
        "model size": modelSize,
        "gpu": gpu_name,
        "epochs": epochs,
        "batch": batch,
        "imgsz": imgsz,
    })
    
    # Train the model
    results = model.train(
        data=data_path, 
        epochs=epochs, 
        imgsz=imgsz,
        batch=batch,    
        flipud=0.0,     # vertical flip
        fliplr=0.0,     # horizontal flip
        mosaic=0.0,     # mosaic augmentation
        mixup=0.0,      # mixup augmentation
        copy_paste=0.0,
        project=project_path,
        name=f"train/{time_now}",
        exist_ok=True
    )
    
    mlflow.log_params({
        "training time": format_duration(time.time() - start_time),
    })
    
    ## Tidak perlu karena sudah otomatis ter-log oleh ultralytics
    # mlflow.log_artifacts(f"results/object-detection/{modelSize}/{time_now}/weights/best.pt", artifact_path="best_model")

mlflow.end_run()

2025/10/04 18:46:07 INFO mlflow.tracking.fluent: Experiment with name 'courtPlay-l' does not exist. Creating a new experiment.


New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.155  Python-3.11.13 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 16311MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/keypoints-v2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l-pose.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=10-04_18-45, nbs=64, nms=Fals

train: Scanning E:\Tennis_Analysis\KeyPoints\datasets\keypoints-v2\train\labels... 2494 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2494/2494 [00:03<00:00, 681.75it/s]


train: New cache created: E:\Tennis_Analysis\KeyPoints\datasets\keypoints-v2\train\labels.cache
val: Fast image access  (ping: 0.10.0 ms, read: 13.94.5 MB/s, size: 181.0 KB)


val: Scanning E:\Tennis_Analysis\KeyPoints\datasets\keypoints-v2\valid\labels... 357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 357/357 [00:00<00:00, 559.85it/s]

...

MLflow: logging run_id(dd6c5e1c2faa41729bb80bc1257f2369) to https://dagshub.com/abiyamf/courtPlay.mlflow
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to results\key-points-v2\yoloyolo11\l\train\10-04_18-45
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


        1/1      10.6G       1.24      3.353     0.2112       1.02      1.581          2       1280: 100%|██████████| 624/624 [05:25<00:00,  1.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:10<00:00,  4.45it/s]


                   all        357        357      0.734      0.616      0.727      0.446      0.666      0.503      0.546      0.178

1 epochs completed in 0.094 hours.
Optimizer stripped from results\key-points-v2\yoloyolo11\l\train\10-04_18-45\weights\last.pt, 53.0MB
Optimizer stripped from results\key-points-v2\yoloyolo11\l\train\10-04_18-45\weights\best.pt, 53.0MB

Validating results\key-points-v2\yoloyolo11\l\train\10-04_18-45\weights\best.pt...
Ultralytics 8.3.155  Python-3.11.13 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 16311MiB)
YOLO11l-pose summary (fused): 199 layers, 26,137,114 parameters, 0 gradients, 90.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:08<00:00,  5.09it/s]


                   all        357        357      0.735      0.623       0.73      0.446      0.668      0.504      0.547      0.178
Speed: 0.5ms preprocess, 18.6ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to results\key-points-v2\yoloyolo11\l\train\10-04_18-45
MLflow: mlflow run still alive, remember to close it using mlflow.end_run()
MLflow: results logged to https://dagshub.com/abiyamf/courtPlay.mlflow
MLflow: disable with 'yolo settings mlflow=False'
🏃 View run 10-04_18-45 at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/4/runs/dd6c5e1c2faa41729bb80bc1257f2369
🧪 View experiment at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/4


## Inference Image

In [19]:
model = YOLO(f"{project_path}/train/{time_now}/weights/best.pt")
list_of_images = ["examples/images/"+i for i in os.listdir("examples/images/") if i.endswith(".jpg")]
results = model(list_of_images, stream=True)
os.makedirs(f"{project_path}/predict/{time_now}/", exist_ok=True)
for result in results:
    
    result.save(filename=f"{project_path}/predict/{time_now}/"+result.path.split("/")[-1])


0: 736x1280 (no detections), 30.2ms
1: 736x1280 (no detections), 30.2ms
2: 736x1280 (no detections), 30.2ms
3: 736x1280 (no detections), 30.2ms
4: 736x1280 (no detections), 30.2ms
5: 736x1280 (no detections), 30.2ms
6: 736x1280 (no detections), 30.2ms
Speed: 3.9ms preprocess, 30.2ms inference, 0.4ms postprocess per image at shape (1, 3, 736, 1280)


## Inference Video

In [17]:
model = YOLO(f"{project_path}/train/{time_now}/weights/best.pt")
list_of_videos = ["examples/videos/"+i for i in os.listdir("examples/videos/") if i.endswith(".mp4")]

# Make sure folder exists
os.makedirs(f"{project_path}/track/", exist_ok=True)

for video in list_of_videos:
    results = model.track(video, save=True, tracker="bytetrack.yaml", project=project_path, name=f"track/{time_now}/", exist_ok=True)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1818) e:\Tennis_Analysis\KeyPoints\examples\videos\video_1.mp4: 736x1280 1 tennis_court, 36.0ms
video 1/1 (frame 2/1818) e:\Tennis_Analysis\KeyPoints\examples\videos\video_1.mp4: 736x1280 1 tennis_court, 29.3ms
video 1/1 (frame 3/1818) e:\Tennis_Analysis\KeyPoints\examples\videos\video_1.mp4: 736x1280 1 tennis_court, 29.4ms
video 1/1 (frame 4/1818) e:\Tennis_Analysis\KeyPoints\examples\videos\video_1.mp4: 736x1280 1 tennis_court, 29.4ms